In [1]:
import climate_functions as cf
import numpy as np
import synthetic_data as sd
import network_builder as nb
import networkx as nx
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
#from fastdtw import fastdtw
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#from dtaidistance import clustering
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
sns.set()
import random
import pandas as pd
from netCDF4 import Dataset
from stldecompose import decompose, forecast
import folium
import math
import scipy.stats as st
#import cufflinks as cf
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='mohammad5617', api_key='yElRsXEjk0sRoqCmfVBn')

/home/mathsys2/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
f_pre = Dataset('precipitation.nc')
pr = f_pre.variables['precip']
lon = f_pre.variables['lon'][:]
lat = f_pre.variables['lat'][:]
pr = np.swapaxes(pr,0,2)

In [40]:
#data = pr[10:22,3:11,:]
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [4]:
r = np.arange(0.1,1,0.05)

In [7]:
data_deseasonal= cf.deseasonal_STL(data)
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)
data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)
MSEds_list = []
r2ds_list = []
MSE_list = []
r2_list = []
MSEdm_list = []
r2dm_list = []
null_list =[]
nullds_list = []
nulldm_list = []
for i in range(200,1,-10):
    R2_ds, MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, error_ds, null_r2_ds,density_ds = cf.model(data_deseasonal,"LR","normal",start=i+300,end=i)
    MSEds_list.append(MSE_ds)
    r2ds_list.append(R2_ds)
    nullds_list.append(null_r2_ds)
    R2, MSE, average_betweennes, average_shortest_path_length, error, null_r2, density = cf.model(data,"LR","normal",start=i+300,end=i)
    MSE_list.append(MSE)
    r2_list.append(R2)
    null_list.append(null_r2)
    R2_dm, MSE_dm, average_betweennes_dm, average_shortest_path_length_dm, error_dm, null_r2_dm, density_dm = cf.model(data_deseasonal_monthly,"LR","normal",start=i+300,end=i)
    MSEdm_list.append(MSE_dm)
    r2dm_list.append(R2_dm)
    nulldm_list.append(null_r2_dm)

In [112]:
np.std(nulldm_list)/math.sqrt(20) * 1.96

0.6149284214664476

In [8]:
f = np.matrix(r2_list).std(axis=0)/math.sqrt(20) * 1.96
r2_list_bounds = np.reshape(f,(18,1))
#r2_list_bounds = r2_list_bounds.tolist()[0]


In [9]:
nullr2 = np.mean(null_list)

In [10]:
nullr2

-1.0600348435858349

In [11]:
nullr2ds = np.mean(nullds_list)

In [12]:
f = np.matrix(r2_list).mean(axis=0)
r2 = np.reshape(f,(18,1))

f1 = np.matrix(r2ds_list).mean(axis=0)
r2_ds = np.reshape(f1,(18,1))

f2 = np.matrix(r2dm_list).mean(axis=0)
r2_dm = np.reshape(f2,(18,1))

r_rev = r[::-1]

r2_upper = r2 + r2_list_bounds
r2_lower = r2 - r2_list_bounds

In [13]:
upper_bound = go.Scatter(
    name='Upper Bound',
    x=r,
    y=r2_upper,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty'
)
trace0 = go.Scatter(
    x = r ,
    y = r2,
    name = 'R2 of normal data',
    mode = 'lines',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,dash = 'dot'),
   # fillcolor='rgba(68, 68, 68, 0.3)',
    #fill='tonexty'
)

lower_bound = go.Scatter(
    name='Lower Bound',
    x=r,
    y=r2_lower,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0)
)
trace1 = go.Scatter(
    x = r,
    y = r2_ds,
    name = 'R2 of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)   
trace2 = go.Scatter(
    x = r,
    y = r2_dm,
    name = 'R2 of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)



d = [trace0, trace1]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              range=[-0.4, 0.8],
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
                            shapes = [
        {
            'type': 'line',
            'x0': 0,
            'y0': nullr2,
            'x1': 1,
            'y1': nullr2,
            'line': {
                'color': 'rgb(255, 0, 0)',
                'width': 0.5,
            },
        }]
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [41]:
R2, MSE, average_betweennes, average_shortest_path_length, error, null_r2, density = cf.model(data,"LR","normal")

data_deseasonal= cf.deseasonal_STL(data)
R2_ds, MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, error_ds, null_r2_ds, density_ds = cf.model(data_deseasonal,"LR","normal")

data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
R2_dsm, MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, error_dsm, null_r2_dsm, density_dm = cf.model(data_deseasonal_monthly,"LR","normal")


In [15]:
trace0 = go.Scatter(
    x = r,
    y = R2,
    name = 'R2 of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = R2_ds,
    name = 'R2 of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = R2_dsm,
    name = 'R2 of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
                            shapes = [
        {
            'type': 'line',
            'x0': 0.35,
            'y0': 0,
            'x1': 0.35,
            'y1': 1,
            'line': {
                'color': 'rgb(0, 0, 0)',
                'width': 0.5,
            },
        }]
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [16]:
trace0 = go.Scatter(
    x = r,
    y = average_shortest_path_length,
    name = 'Average Shortest Path of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = average_shortest_path_length_ds,
    name = 'Average Shortest Path of deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
)    
d = [trace0, trace1]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds'),
              yaxis = dict(title = 'Average Shortest Path'),
              shapes = [
        {
            'type': 'line',
            'x0': 0.35,
            'y0': 0,
            'x1': 0.35,
            'y1': 3.5,
            'line': {
                'color': 'rgb(0, 0, 0)',
                'width': 0.5,
            },
        }]
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [17]:
trace0 = go.Scatter(
    x = r,
    y = average_betweennes,
    name = 'Average Betweennes of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = average_betweennes_ds,
    name = 'Average Betweennes of deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
)    
d = [trace0, trace1]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds'),
              yaxis = dict(title = 'Average Betweennes'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [43]:
density = 1 - np.array(density)
density_ds = 1 - np.array(density_ds)

In [44]:
trace0 = go.Scatter(
    x = r,
    y = density,
    name = 'confidence of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2)
)
trace1 = go.Scatter(
    x = r,
    y = density_ds,
    name = 'confidence of deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,)
)    
d = [trace0, trace1]
    

layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Confidence',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000')
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [28]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [30]:
R2, MSE, average_betweennes, average_shortest_path_length, error, null_r2, density = cf.model(data,"NN","normal")

data_deseasonal= cf.deseasonal_STL(data)
R2_ds, MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, error_ds, null_r2_ds, density_ds = cf.model(data_deseasonal,"NN","normal")

data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
R2_dsm, MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, error_dsm, null_r2_dsm, density_dm = cf.model(data_deseasonal_monthly,"NN","normal")


In [31]:
trace0 = go.Scatter(
    x = r,
    y = R2,
    name = 'R2 of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = R2_ds,
    name = 'R2 of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = R2_dsm,
    name = 'R2 of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [32]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [34]:
R2, MSE, average_betweennes, average_shortest_path_length, error, null_r2, density = cf.model(data,"RF","normal")

data_deseasonal= cf.deseasonal_STL(data)
R2_ds, MSE_ds, average_betweennes_ds, average_shortest_path_length_ds, error_ds, null_r2_ds, density_ds = cf.model(data_deseasonal,"RF","normal")

data_deseasonal_monthly= cf.deseasonal_monthly_anomaly(data)
R2_dsm, MSE_dsm, average_betweennes_dsm, average_shortest_path_length_dsm, error_dsm, null_r2_dsm, density_dm = cf.model(data_deseasonal_monthly,"RF","normal")

In [35]:
trace0 = go.Scatter(
    x = r,
    y = R2,
    name = 'R2 of normal data',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,)
)
trace1 = go.Scatter(
    x = r,
    y = R2_ds,
    name = 'R2 of STL deseasonal data',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 2,
        dash = 'dot')
)    

trace2 = go.Scatter(
    x = r,
    y = R2_dsm,
    name = 'R2 of monthly deseasonal data',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

d = [trace0, trace1, trace2]
    
layout = dict(title = 'Comparison of normal and deseasonal data',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'R-squared',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [36]:
data = pr[16:23,10:16,:]
data = cf.unflatten(data)
data = np.swapaxes(data,0,1)

In [37]:
r = np.arange(0.1,1,0.05)
df = cf.feature_importance(data)

In [38]:
trace0 = go.Scatter(
    x = r,
    y = df['degree'],
    name = 'degree',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
    dash = 'solid')
)
 
trace2 = go.Scatter(
    x = r,
    y = df['closseness'],
    name = 'closseness',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = df['kcore'],
    name = 'kcore',
    line = dict(
        color = ('rgb(202, 125, 68)'),
        width = 4,
        dash = 'dot')
)
trace4 = go.Scatter(
    x = r,
    y = df['betweeness'],
    name = 'betweeness',
    line = dict(
        color = ('rgb(105, 184, 68)'),
        width = 4,
        dash = 'dashdot')
)
trace5 = go.Scatter(
    x = r,
    y = df['pagerank'],
    name = 'pagerank',
    line = dict(
        color = ('rgb(3, 1, 5)'),
        width = 4,
        dash = 'longdashdot')
)
trace6 = go.Scatter(
    x = r,
    y = df['eigenvector'],
    name = 'eigenvector',
    line = dict(
        color = ('rgb(239, 38, 205)'),
        width = 4,
        dash = 'longdash')
)
trace7 = go.Scatter(
    x = r,
    y = df['clustering'],
    name = 'clustering',
    line = dict(
        color = ('rgb(248, 215, 34)'),
        width = 4,
        dash = 'dash')
)
d = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title = 'Features Importance based on Random Forest',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Importance',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [39]:
data_deseasonal= cf.deseasonal_STL(data)
df = cf.feature_importance(data_deseasonal)

trace0 = go.Scatter(
    x = r,
    y = df['degree'],
    name = 'degree',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 2,
    dash = 'solid')
)
 
trace2 = go.Scatter(
    x = r,
    y = df['closseness'],
    name = 'closseness',
    line = dict(
        color = ('rgb(33, 164, 242)'),
        width = 4,
        dash = 'dash')
)

trace3 = go.Scatter(
    x = r,
    y = df['kcore'],
    name = 'kcore',
    line = dict(
        color = ('rgb(202, 125, 68)'),
        width = 4,
        dash = 'dot')
)
trace4 = go.Scatter(
    x = r,
    y = df['betweeness'],
    name = 'betweeness',
    line = dict(
        color = ('rgb(105, 184, 68)'),
        width = 4,
        dash = 'dashdot')
)
trace5 = go.Scatter(
    x = r,
    y = df['pagerank'],
    name = 'pagerank',
    line = dict(
        color = ('rgb(3, 1, 5)'),
        width = 4,
        dash = 'longdashdot')
)
trace6 = go.Scatter(
    x = r,
    y = df['eigenvector'],
    name = 'eigenvector',
    line = dict(
        color = ('rgb(239, 38, 205)'),
        width = 4,
        dash = 'longdash')
)
trace7 = go.Scatter(
    x = r,
    y = df['clustering'],
    name = 'clustering',
    line = dict(
        color = ('rgb(248, 215, 34)'),
        width = 4,
        dash = 'dash')
)
d = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(title = 'Features Importance based on Random Forest',
              xaxis = dict(title = 'Thresholds',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=0.05,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              yaxis = dict(title = 'Importance',
              showticklabels=True,
              tickmode='linear',
              ticks='outside',
              tick0=0,
              dtick=1,
              ticklen=8,
              tickwidth=4,
              tickcolor='#000'),
              )

fig = dict(data=d, layout=layout)
py.iplot(fig, filename='styled-line')

In [8]:
trace = go.Scatter(
    x = r,
    y = MSE
)

data = [trace]
layout = {}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='line-mode')